In [54]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="deepseek-r1-distill-llama-70b")

In [55]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [56]:
system_messages = "You are a helpful assistant"

In [57]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_messages),
        ("human", 
         "Analyze the following student response and classify its correctness. "
         "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
         "Student Response: {response}\n\n"
         "Provide the classification following this format:\n{format_instruction}")
    ]
).partial(format_instruction=parser.get_format_instructions())

In [58]:
classifier_chain = chat_prompt | model | parser

In [59]:
classifier_chain.invoke(
    {
        'response': "This sum of 2+2=5"
    }
)

StuOutputCorrectness(sentiment='incorrect')

## ***Adaptive Prompt With Messages History***

In [60]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Analyze the following student response and classify its correctness."
         "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
         "Student Response: {response}\n\n"
         "Provide the classification following this format:\n{format_instruction}"),
        MessagesPlaceholder(variable_name='messages')
    ]
).partial(format_instruction=parser.get_format_instructions())

In [61]:
classifier_chain = chat_prompt | model | parser

In [62]:
## List of messages
messages = [
                HumanMessage(content="solve this problem and 2+2=0?"),
                AIMessage(content="This is a addition problem and think about you have tow apple and your friend also give 2 more apple now how many apple you have. Its a hints to solve this problem."),
            ]

In [63]:
classifier_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)

StuOutputCorrectness(sentiment='partially_correct')

## Add Branch

In [64]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "Student gave the correct response and the response is {response}. "
         "Now give an adaptive prompt so the model can generate only the best response for the next step. "
         "Also analyze previous message history — it will help generate a better adaptive prompt."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

prompt_partially_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         your task is generate a best adaptive prompt. And this adaptive prompt goes to my main model as the system prompt. Using this prompt the main model will be generated the elaborated response you just generate one the adaptive prompt only. Think you just help to main model as a instructor like a boss.
         for generate one best adaptive prompt you can analysis the previous chat and current response also.
         Student gave a response which is partially correct and the response is {response}. 
         """
         ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

prompt_incorrect = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "Student gave a totally incorrect response and the response is {response}. "
         "Analyze the student’s misunderstanding — maybe they need clarification on core concepts. "
         "Provide elaborated hints or ask guiding questions. "
         "Use previous message history to craft the best adaptive prompt for the next step."),
        MessagesPlaceholder(variable_name="messages")
    ]
)


In [65]:
str_parser = StrOutputParser()

In [66]:
branch_chain = RunnableBranch(
    (lambda x: x["sentiment"] == 'correct', prompt_correct | model | str_parser),
    (lambda x: x["sentiment"] == 'partially_correct', prompt_partially_correct | model | str_parser),
    (lambda x: x["sentiment"] == 'incorrect', prompt_incorrect | model | str_parser),
    RunnableLambda(lambda x: "Couldn't determine sentiment.")
)


In [67]:
from langchain_core.runnables import RunnableLambda

# Wrap classifier_chain to return full input + parsed sentiment
def enrich_with_sentiment(input: dict) -> dict:
    sentiment_result = classifier_chain.invoke(input)
    print(sentiment_result)
    return {
        **input,  # includes 'messages' and 'response'
        'sentiment': sentiment_result.sentiment
    }

enriched_chain = RunnableLambda(enrich_with_sentiment)


In [68]:
final_chain = enriched_chain | branch_chain

In [69]:
adaptive_prompt = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt)

sentiment='correct'
 Now solve the problem 2+2=?
</think><think>
Okay, so the user is trying to solve the problem 2+2=?. They mentioned something about apples, like if I have two apples and my friend gives me two more, how many do I have? That makes sense because adding apples is a common way to explain addition. 

But wait, in their initial message, they said maybe they have 3 apples or 4 and they're confused. So I guess they're not sure if it's 3 or 4. Then they said 2+2=4. Hmm, but then they asked why is 2+2=0? That's confusing because 2+2 is definitely 4, not 0.

I think the user might be mixing up different math concepts. Maybe they heard somewhere that 2+2 can be something else, like in binary or another context, but in basic arithmetic, it's always 4. Or maybe they're just trying to trick me, but I don't think so. They seem genuinely confused.

I should probably explain that in regular addition, 2+2 equals 4. Using the apple example helps visualize it: starting with two apples a

In [70]:
adaptive_prompt2 = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt2)

sentiment='correct'
 So 2+2=4. So 2+2=4 is correct. But if you think 2+2=0 this is wrong. So the correct answer is 4.

Okay, so the user is asking about 2+2=0 and seems confused. They mentioned having apples but are unsure if it's 3 or 4. Earlier, they thought the answer was 4, but now they're considering 0. Hmm, maybe they're overcomplicating it. I should guide them back to the basics.

I need to create an adaptive prompt that helps the model address their confusion. Let me break it down: acknowledge their uncertainty, use the apple analogy again, explain why 4 is correct, and correct the misunderstanding about 0. Keep it clear and reassuring so they feel confident in the answer.

Alright, let me structure the prompt to cover these points. Make sure it's concise and leads the model to provide a supportive and educational response.
</think>

Certainly! Here's an adaptive prompt based on the previous message history:

---

**Adaptive Prompt:**  
"Recognize that the user is confused abou

## get structure adaptive prompt

In [71]:
class AdaptivePrompt(BaseModel):
    adaptive_prompt: str = Field(description="The final adaptive prompt to guide the main model.")


In [72]:
adaptive_parser = PydanticOutputParser(pydantic_object=AdaptivePrompt)

prompt_partially_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         Your task is to generate one adaptive prompt only. This prompt will be used as the system prompt for the main model. 
         Think of yourself as a supervisor providing the best instruction for the AI tutor.

         You can analyze the previous chat history and the student’s latest response.

         Student gave a response which is partially correct: {response}

         Return your result in the following format:
         {format_instructions}
         """
         ),
        MessagesPlaceholder(variable_name="messages")
    ]
).partial(format_instructions=adaptive_parser.get_format_instructions())



In [73]:
branch_chain = RunnableBranch(
    (lambda x: x["sentiment"] == 'correct', prompt_correct | model | adaptive_parser),
    (lambda x: x["sentiment"] == 'partially_correct', prompt_partially_correct | model | adaptive_parser),
    (lambda x: x["sentiment"] == 'incorrect', prompt_partially_correct | model | adaptive_parser),
    RunnableLambda(lambda x: {"adaptive_prompt": "Couldn't determine sentiment."})
)

In [74]:
from langchain_core.runnables import RunnableLambda

# Wrap classifier_chain to return full input + parsed sentiment
def enrich_with_sentiment(input: dict) -> dict:
    sentiment_result = classifier_chain.invoke(input)
    print(sentiment_result)
    return {
        **input,  # includes 'messages' and 'response'
        'sentiment': sentiment_result.sentiment
    }

enriched_chain = RunnableLambda(enrich_with_sentiment)

In [75]:
final_chain = enriched_chain | branch_chain

In [76]:
adaptive_prompt3 = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt3)

sentiment='incorrect'
adaptive_prompt="Let's solve the problem step by step. If you have 2 apples and your friend gives you 2 more, you now have a total of 4 apples. So, 2 + 2 equals 4, not 0. Addition is about combining groups, so when you add 2 and 2, you get 4."


In [77]:
final_chain.get_graph().print_ascii()

+-----------------------------+  
| enrich_with_sentiment_input |  
+-----------------------------+  
                *                
                *                
                *                
    +--------------------+       
    | ChatPromptTemplate |       
    +--------------------+       
                *                
                *                
                *                
          +----------+           
          | ChatGroq |           
          +----------+           
                *                
                *                
                *                
    +----------------------+     
    | PydanticOutputParser |     
    +----------------------+     
                *                
                *                
                *                
          +--------+             
          | Branch |             
          +--------+             
                *                
                *                
              